In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/complete_spinetta.csv', sep=';')
df.sample(5)

,disc_number,duration_ms,name,track_number,album,artist,author,lyrics_url,lyrics
222,1,205986,Yo Miro Tu Amor,4,Para Los Arboles,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-yo-mi...,"En tanto que la lluvia... ( así ) Lava, mis ..."
37,1,89773,Jingle,19,El 1er Álbum Más los Singles,Almendra,Luis Alberto Spinetta,https://genius.com/Almendra-jingle-lyrics,Vuelve del cuento feroz Su sombra está por s...
215,1,356500,Atado A Tu Frontera,9,Pan,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-atado...,En la inconsolable noche tu voz Aullando me ...
307,1,242750,Yo Quiero Ver Un Tren,2,Mondo Di Cromo,Luis Alberto Spinetta,Luis Alberto Spinetta,https://genius.com/Luis-alberto-spinetta-yo-qu...,Yo nunca me imaginé regresar a mi tiempo de ...
106,1,165893,Siguiendo Los Pasos Del Maestro,5,Los Niños Que Escriben En El Cielo,Spinetta Jade,Luis Alberto Spinetta,https://genius.com/Spinetta-jade-siguiendo-los...,-


## MARKOV

__Get list of all lyrics of non instrumental songs__

In [3]:
lyrics = df[df['lyrics'] != '-'].lyrics.values

In [4]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """

    # Initialize the beginning of our chain.
    chain = {
        (None, "<START>"): []
    }

    for lyric in lyrics:
        # Replace newline characters with our tag.
        lyric_newlines = lyric.replace('\n', ' <N> ')
        # Create a tuple representing the most recent (current) bigram.
        last_2 = (None, "<START>")
        for word in lyric_newlines.split():
            # Add the word as one that follows the current bigram.
            chain[last_2].append(word)
            # Shift the current bigram to account for the newly added word.
            last_2 = (last_2[1], word)
            if last_2 not in chain:
                chain[last_2] = []
        chain[last_2].append("<END>")

    return chain

In [5]:
# Train a Markov Chain over all of Logic's lyrics.
chain = train_markov_chain(lyrics)

In [6]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    Returns:
      A string representing the randomly generated song.
    """

    # a list for storing the generated words
    words = []
    # generate the first word
    word = random.choice(chain[(None, "<START>")])
    words.append(word)

    # Begin with the first bigram in our chain.
    last_2 = (None, "<START>")
    while words[-1] != "<END>":
        # Generate the next word.
        word = random.choice(chain[last_2])
        words.append(word)
        # Shift the current bigram to account for the newly added word.
        last_2 = (last_2[1], words[-1])

    # Join the words together into a string with line breaks.
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [13]:
print(generate_new_lyrics(chain))

Yo Entre las miserias Y el concierto del aire 
 Y dejé mis zapatos para un niño nace 
 las nubes vendrán.... Trayendo mensajes... En el nadie de la city 
 y como un canto 
 canta porque es vida 
 al quebrar un llanto Luna y espera de serrazón Pronto esto cambiará Es un espejo de cristal


In [14]:
print(generate_new_lyrics(chain))

Gigante ¡Oh!, ¿quien te puede ver? 
 cuando la luna en diamantes... Es un milagro te haría salir Del barro no volverá Adentro queda un cuerpo Entre tus lágrimas Y el durazno partido Ya sangrando está bajo el sol a sol 
 sobre todo niños 
 si alguna vez desde mis pies Bailando porque soy mestizo Voy al norte de nada Donde sopla el viento te arrebatará... Esa hoja escrita Con tu molinete Que los amos No descansan... Ya no me digas que se inyectan ciudades ¿y qué? ¿acaso no marchitan los navíos y las rosas en los jardines nena Esta es mi feroz canción Para dormir en el cosmos Ahí va el Capitán Beto por el mal Se ríe el niño Y así yo en la canción así Una esquina desigual y un rayo al amanecer. 
 
 Tengo fiebre 
 cincuenta y pico 
 tengo un vientre blanco Dale tibia leche de tu alma Para impulsar a este mágico Y misterioso mundo Ah !, luego en el más allá Ludmila Ludmila yo veo Tus dedos de mimbre Quiero saber porque lloran... Ti, ti, ti, ti, ti, ti, ti


In [16]:
print(generate_new_lyrics(chain))

Me Caído de un vuelo limpio Entre túneles de luz Y esta vez no aceptes que otro amor vendrá. 
 
 Un sueño de alguien que se da... Solo refleja en edificios mudos... Contra el muro Al que no duermen No hay amigos, solo una Todo el mundo es extravío, 
 en sí La fuerza del fuego La voz de Cris me llega igual Y veo la cara de aquella canción sin fin Y además Dale gracias por estar cerca de ti Todo se ha movido con oscuridad Todo este tiempo Ya sé, ya no existe aquí Un congelado amanecer Tiñe de blanco hasta mi Vamos a mi! A mi! Si las flores que esperan... Condición de fulgor en algún planeta, en algún lugar 
 con las manos ante Bosnia Algo se está cayendo Es ese cielo Te voy a estar En este cementerio Qué calor hará sin vos En mi nunca la luz del amanecer... ¡Oh mi amor Esta vez solo quiero una sola cosa Dame tu luz Veo así a través de un grito De inocencia... ¿es que nunca calla Solo se ve se ama se pierde en el mar, el mar 
 Algo se acumula en mí Y la distancia no me mires dicta tu sole

In [23]:
print(generate_new_lyrics(chain))

Instant-táneas Nena, tu cabeza va a estallar Ondas en aire Ondas en aire Un columpio fabuloso en el rebaño del pastor...


In [24]:
print(generate_new_lyrics(chain))

Márcame, El mandarín Se descentró Sus manos ya no toca un libro de sal, 
 valle de eterna sal... 
 hoyo en la leña... 
 
 Un destello en el aire En la rapsodia típica de Bosnia Donde el mundo tu ser te dolerá al fin A la luna Viejo roble del camino Tus hojas siempre se la llevan detenida Como a una estrella en su luz El cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Cielo en blanco enorme lejísimo Lejísimo lejísimo Nosotros adentro Nosotros adentro Yo quiero ver Si acaso las nubes vendrán.... Trayendo mensajes...
